# Lista 3 de Introdução à Programação e Ciência de Dados

Professor: Rafael de Pinho

Monitor: Sillas Rocha

Aluno: Iago Dantas Figueirêdo

## Instruções Gerais
- Cada questão deve ser implementada em um módulo separado, com funções reutilizáveis.
- Documente todas as funções com docstrings no estilo PEP 257 e use type hints.
- Utilize apenas as bibliotecas padrão do Python e o ```NumPy```. Bibliotecas como ```pandas```, ```scipy``` e similares não são permitidas.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from simulations import (
    simular_precos,
    calc_retornos_simples,
    calc_retornos_log,
    sma,
    rolling_std,
)

## Parte 1: Simulação de Preços e Análise de Retornos

Esta primeira parte desenvolve a simulação de dados, esse tipo de simulação possui diversos usos. Implemente todos estes exercícios no módulo "simulations.py".

### 1. Simulação de Série de Preços com Ruído Gaussiano
**Função**: simular_precos(S0: float, sigma: float, days: int) -> np.ndarray

**Descrição**: Gere uma série temporal de preços de ações de forma simplificada:

$$
S_{t+1}=S_t+\varepsilon_t, \quad \varepsilon_t \sim \mathcal{N}\left(0, \sigma^2\right) .
$$


A função deve retornar um ```np.ndarray``` de tamanho ```days + 1``` , onde o primeiro elemento é S0 e, a cada passo, soma-se um ruído normal de desvio padrão ```sigma```.

**Parâmetros**:
- ```S0```: preço inicial positivo.
- ```sigma```: desvio padrão do ruído (volatilidade).
- ```days```: número de dias a simular.

**Retorno**: ```np.ndarray``` com preços simulados.

In [3]:
# Exemplo de uso da função simular_precos

S0 = 100
sigma = 1
days = 21

precos = simular_precos(S0, sigma, days)

print(f"Preços simulados para {days} dias:")
for i, preco in enumerate(precos):
    print(f"Dia {i}: {preco:.2f}")

Preços simulados para 21 dias:
Dia 0: 100.00
Dia 1: 100.50
Dia 2: 100.36
Dia 3: 101.01
Dia 4: 102.53
Dia 5: 102.30
Dia 6: 102.06
Dia 7: 103.64
Dia 8: 104.41
Dia 9: 103.94
Dia 10: 104.48
Dia 11: 104.02
Dia 12: 103.55
Dia 13: 103.79
Dia 14: 101.88
Dia 15: 100.16
Dia 16: 99.59
Dia 17: 98.58
Dia 18: 98.89
Dia 19: 97.99
Dia 20: 96.57
Dia 21: 98.04


### 2. Cálculo de Retornos Simples e Logarítmicos

**Função**: ```calc_retornos_simples(prices: np.ndarray) -> np.ndarray```

**Descrição**: Dado um vetor de preços $P=\left[P_0, P_1, \ldots, P_n\right]$, calcule os retornos simples diários:

$$
r_t=\frac{P_t-P_{t-1}}{P_{t-1}}, \quad t=1, \ldots, n
$$

Retorne um vetor de dimensão $n$.

---

**Função**: ```calc_retornos_log(prices: np.ndarray) -> np.ndarray```

**Descrição**: Para o mesmo vetor de preços $P$, calcule os log-retornos:

$$
r_t^{\log }=\ln \left(P_t / P_{t-1}\right), \quad t=1, \ldots, n .
$$


Retorne um vetor de dimensão $n$.

In [4]:
# Exemplo de uso das funções calc_retornos_simples e calc_retornos_log

prices = np.array([100, 102, 101, 105, 107])

retornos_simples = calc_retornos_simples(prices)
retornos_log = calc_retornos_log(prices)

print("\nRetornos Simples:")
for i, retorno in enumerate(retornos_simples):
    print(f"Dia {i+1}: {retorno:.4f}")
print("\nRetornos Logarítmicos:")
for i, retorno in enumerate(retornos_log):
    print(f"Dia {i+1}: {retorno:.4f}")


Retornos Simples:
Dia 1: 0.0200
Dia 2: -0.0098
Dia 3: 0.0396
Dia 4: 0.0190

Retornos Logarítmicos:
Dia 1: 0.0198
Dia 2: -0.0099
Dia 3: 0.0388
Dia 4: 0.0189


### 3. Indicadores Móveis: Média e Volatilidade

**Função**: ```sma(returns: np.ndarray, window: int) -> np.ndarray```

**Descrição**: Implemente a Média Móvel Simples (SMA) para um vetor de retornos $r=\left[r_1, \ldots, r_n\right]$. Para cada índice $t$ a partir de $t=$ window, calcule:

$$
\mathrm{SMA}_t=\frac{1}{\text { window }} \sum_{i=t-\text{window}+1}^t r_i .
$$


Retorne um vetor de tamanho $n-\text{window}+1$.

---

**Função**: ```rolling_std(returns: np.ndarray, window: int, days_size: int = 0) -> np.ndarray```

**Descrição**: Calcule o desvio padrão móvel para o vetor de retornos $r$. Para cada $t \geq \text{window}$, defina:

$$
\bar{r}_t = \frac{1}{\text{window}} \sum_{i=t-\text{window}+1}^t r_i, \quad \sigma_t=\sqrt{\frac{1}{\text{window - days\_size }} \sum_{i=t-\text{window}+1}^t\left(r_i-\bar{r}_t\right)^2} .
$$

Retorne um vetor de tamanho $n-\text{window}+1$ . O parâmetro opcional ```days_size``` permite ajustar a normalização (por exemplo, para séries anuais, mensais, etc.).

In [5]:
# Exemplo de uso das funções sma e rolling_std

returns = np.array([0.01, 0.02, -0.01, 0.03, 0.02, -0.02, 0.01])
window = 4

sma_result = sma(returns, window)
rolling_std_result = rolling_std(returns, window)

print("\nSMA (Média Móvel Simples):")
for i, sma_value in enumerate(sma_result):
    print(f"Dia {i + window - 1}: {sma_value:.4f}")
print("\nDesvio Padrão Móvel:")
for i, std_value in enumerate(rolling_std_result):
    print(f"Dia {i + window - 1}: {std_value:.4f}")


SMA (Média Móvel Simples):
Dia 3: 0.0125
Dia 4: 0.0150
Dia 5: 0.0050
Dia 6: 0.0100

Desvio Padrão Móvel:
Dia 3: 0.0148
Dia 4: 0.0150
Dia 5: 0.0206
Dia 6: 0.0187
